# Metrics for Evaluating Denoising Performance
This jupyter notebook provides code for calculating image quality metrics used in the manuscript, including Peak Signal-to-Noise Ratio (PSNR), Structural Similarity Index (SSIM), and Signal-to-Noise Ratio (SNR).

In [ ]:
from PIL import Image
import glob
import os
import numpy as np
import cv2 as cv
import math
import matplotlib.pyplot as plt
import natsort
np.random.seed(2222)

In [ ]:
def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))

def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2
    
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv.filter2D(img1, -1, window)[5:-5, 5:-5] 
    mu2 = cv.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()

def calculate_ssim(img1, img2):
    # img1 and img2 have range [0, 255]
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

def calculate_snr(img1, img2):
    # cross-correlation based SNR calculation
    # Reference: Nature 256, 376-379 (1975)
    img1_1 = img1
    img2_1 = img2
    img1 = Image.open(img1)
    img2 = Image.open(img2)
    mean1 = np.mean(img1)
    mean2 = np.mean(img2)
    r = np.mean((img1 - mean1) * (img2 - mean2)) / np.sqrt(np.mean(np.power(img1 - mean1, 2)) * np.mean(np.power(img2 - mean2, 2)))
    if r < 0:
        print(img1_1, img2_1, r)
    return 10 * np.log10(r / (1 - r))

## Evaluate image quality metrics of simulated dataset
Calculate PSNR and SSIM for simulated datasets, both before and after denoising, using clean images as reference. Then, plot the frame index against PSNR and SSIM for visualization.

In [ ]:
clean_dir = '/path/to/ground_truth_dataset/' # clean images
noisy_dir = '/path/to/noisy_dataset/' # noisy images
denoised_dir = '/path/to/denoised_dataset/' # denoised images, including results from other methods

clean_list = glob.glob(clean_dir + "*.tif")
noisy_list = glob.glob(noisy_dir + "*.tif")
denoised_list = glob.glob(denoised_dir + "*.tif")

In [ ]:
psnr_noisy = []
ssim_noisy = []
psnr_denoised = []
ssim_denoised = []

for i in range(len(clean_list)):
    clean_image = np.array(Image.open(clean_list[i]))
    noisy_image = np.array(Image.open(noisy_list[i]))
    denoised_image = np.array(Image.open(denoised_list[i]))

    # Normalize image intensities to the range [0, 255]
    clean_image= (clean_image - clean_image.min()) * 255.0 / (clean_image.max() - clean_image.min())
    clean_image = np.round(clean_image)
    noisy_image = (noisy_image - noisy_image.min()) * 255.0 / (noisy_image.max() - noisy_image.min())
    noisy_image = np.round(noisy_image)
    denoised_image = (denoised_image - denoised_image.min()) * 255.0 / (denoised_image.max() - denoised_image.min())
    denoised_image = np.round(denoised_image)

    # Calculate PSNR and SSIM for noisy images compared to the clean reference
    psnr_noisy.append(
        calculate_psnr(clean_image, np.flipud(noisy_image))
    )
    ssim_noisy.append(
        calculate_ssim(clean_image, np.flipud(noisy_image))
    )

    # Calculate PSNR and SSIM for denoised images compared to the clean reference
    psnr_denoised.append(
    calculate_psnr(clean_image, denoised_image)
    )
    ssim_denoised.append(
        calculate_ssim(clean_image, denoised_image)
    )
psnr_noisy = np.asarray(psnr_noisy)
ssim_noisy = np.asarray(ssim_noisy)
psnr_denoised = np.asarray(psnr_denoised)
ssim_denoised = np.asarray(ssim_denoised)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (14, 5))
ax[0].plot(psnr_noisy)
ax[0].set_ylabel('Peak signal-to-noise ratio (dB)')
ax[0].set_xlabel('Frame index')

ax[1].plot(ssim_noisy)
ax[1].set_ylabel('SSIM')
ax[1].set_xlabel('Frame index')
plt.show()

print('mean PSNR = ' + str(np.mean(psnr_noisy)) + ' +- ' + str(np.std(psnr_noisy)))
print('mean SSIM = ' + str(np.mean(ssim_noisy)) + ' +- ' + str(np.std(ssim_noisy)))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (14, 5))
ax[0].plot(psnr_denoised)
ax[0].set_ylabel('Peak signal-to-noise ratio (dB)')
ax[0].set_xlabel('Frame index')

ax[1].plot(ssim_denoised)
ax[1].set_ylabel('SSIM')
ax[1].set_xlabel('Frame index')
plt.show()

print('mean PSNR = ' + str(np.mean(psnr_denoised)) + ' +- ' + str(np.std(psnr_denoised)))
print('mean SSIM = ' + str(np.mean(ssim_denoised)) + ' +- ' + str(np.std(ssim_denoised)))

## Evaluate image quality metrics of real datasets
Calculate SNR for real datasets using only the images within each dataset, and save the results as text files.

In [ ]:
noisy_dir = '/path/to/noisy_dataset/' # noisy images
denoised_dir = '/path/to/denoised_dataset/' # denoised images, including results from other methods
snr_dir = '/path/to/snr_results_files/' # snr results

noisy_list = natsort.natsorted(glob.glob(noisy_dir + "*.tif"))
denoised_list = natsort.natsorted(glob.glob(denoised_dir + "*.tif"))

In [ ]:
num_img = len(noisy_list)
snr_noisy = np.zeros(num_img + 1)
snr_denoised = np.zeros(num_img + 1)

# Calculate SNR for noisy and denoised images
for i in range(num_img - 1):
    snr_noisy[i + 2] = calculate_snr(noisy_list[i], noisy_list[i + 1])
    snr_denoised[i + 2] = calculate_snr(denoised_list[i], denoised_list[i + 1])

# Calculate mean and standard deviation of SNR
snr_noisy[0] = np.mean(snr_noisy[2:])  # Mean SNR (first line)
snr_noisy[1] = np.std(snr_noisy[2:])   # Standard deviation of SNR (second line)

snr_denoised[0] = np.mean(snr_denoised[2:])
snr_denoised[1] = np.std(snr_denoised[2:])

# Save results to text files
np.savetxt(os.path.join(snr_dir, 'snr_noisy.txt'), snr_noisy, fmt="%.6f")
np.savetxt(os.path.join(snr_dir, 'snr_denoised.txt'), snr_denoised, fmt="%.6f")

print('mean SNR of noisy images = ' + str(snr_noisy[0]) + ' +- ' + str(snr_noisy[1]))
print('mean SNR of denoised images = ' + str(snr_denoised[0]) + ' +- ' + str(snr_denoised[1]))